In [1]:
import mxnet as mx
from mxnet import nd
from mxnet import autograd
from mxnet import gluon
import numpy as np

In [2]:
ctx = mx.cpu()

In [3]:
import json

with open("./data/train_data_50000.json") as json_data:
    train_data = json.load(fp=json_data)
    
with open("./data/test_data_50000.json") as json_data:
    test_data = json.load(fp=json_data)

train_X = np.array(train_data['data'])
train_Y = np.array(train_data['label'])

test_X = np.array(test_data['data'])
test_Y = np.array(test_data['label'])

In [4]:
batch_size = 200
num_inputs = 64
num_outputs = 2
train_data = mx.io.NDArrayIter(train_X,train_Y ,
                               batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(test_X, test_Y,
                               batch_size, shuffle=True)

In [5]:
num_hidden = 256
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

In [6]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [7]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [8]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})


In [9]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [10]:
epochs = 40
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
            cross_entropy.backward()
        trainer.step(data.shape[0])
        if i == 0:
            moving_loss = nd.mean(cross_entropy).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(cross_entropy).asscalar()

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" %
          (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 0.677382715461, Train_acc 0.5994, Test_acc 0.13634
Epoch 1. Loss: 0.66288332207, Train_acc 0.60766, Test_acc 0.11466
Epoch 2. Loss: 0.658119773957, Train_acc 0.61322, Test_acc 0.10722
Epoch 3. Loss: 0.654353553126, Train_acc 0.61816, Test_acc 0.10558
Epoch 4. Loss: 0.65041174969, Train_acc 0.6234, Test_acc 0.10602
Epoch 5. Loss: 0.646193166805, Train_acc 0.62984, Test_acc 0.10818
Epoch 6. Loss: 0.641675029941, Train_acc 0.63526, Test_acc 0.1112
Epoch 7. Loss: 0.63696968901, Train_acc 0.64174, Test_acc 0.11702
Epoch 8. Loss: 0.632123405727, Train_acc 0.64764, Test_acc 0.13026
Epoch 9. Loss: 0.627179627871, Train_acc 0.65474, Test_acc 0.14478
Epoch 10. Loss: 0.622283751739, Train_acc 0.65976, Test_acc 0.1608
Epoch 11. Loss: 0.617577099847, Train_acc 0.66644, Test_acc 0.17588
Epoch 12. Loss: 0.613101373794, Train_acc 0.67242, Test_acc 0.19374
Epoch 13. Loss: 0.608835831395, Train_acc 0.67808, Test_acc 0.21464
Epoch 14. Loss: 0.604724415204, Train_acc 0.68194, Test_acc 0.233